# Intro to git for Data Scientists - Part II

# Chapter II

* How to track changes of a file in a repository (log, show, diff)
* How to maintain different versions of a file without going crazy (branch, checkout)
* Time-travel with git (revert, checkout, reset)

**Warning**: This tutorial assumes you have already completed part 0 of this tutorial, and have run all the cells in that notebook *exactly once*

## How to track changes of a file in a repository (log, show, diff)

In the first part of this tutorial we have created a repository and added a few commits to it. Sometimes we will want git to remind us of the latest changes we have performed in our repository - this is where `git log` comes handy: 

In [ ]:
%%bash
cd my_first_repo
git log --graph

**Exercise 1.0**: Show the commit history only for the file (aka. path) `test.py`. 

**Exercise 1.1**: Show the commit history only for the last two commits.

**Exercise 1.2**: Show the commit history only for the first two commits.

*Hint*: Check out [git log's documentation](https://git-scm.com/docs/git-log)

Perhaps we also want to know what exactly we did in a given commit. `git show` is here to help us:

In [ ]:
%%bash
cd my_first_repo
# This will fail because this commit sha is from my repo, not your repo. Update it to the sha of the commit you want to show!
git show 33edc85477fb9db2a67e279b7b031d0a69a05002

### Diffing is the name of the game

Let's spice it up a little. We will now experiment with the `git diff` tool which allows us to understand what exactly changed between two commits, the staged changes and HEAD or other variations thereof.

Before doing that, let's add some meat to our skinny repository! For the next few steps we will work with [an example of Machine Learning code in Python from Scikit-learn's](https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html#sphx-glr-auto-examples-compose-plot-compare-reduction-py) library of examples. This means that you will **need to install scikit-learn v0.20.3 or compatible if you don't have it yet.**

In [ ]:
%%bash
rm -f my_first_repo/plot_compare_reduction.py*
# This script (plot_compare_reduction.py) has been obtained via: 
# wget https://scikit-learn.org/stable/_downloads/plot_compare_reduction.py -P resources
# A local copy is held in case sklearn's version changes
cp resources/plot_compare_reduction.py my_first_repo/
cd my_first_repo
git add plot_compare_reduction.py
git commit -m 'checking scikit learn example into our repo'

Our first step will be to clean up the example to remove the part corresponding to "Caching transformers within a `Pipeline`":

In [ ]:
%%bash
cd my_first_repo
# We will remove all code after line 91
# If you're wondering why redirect and then mv, cf: https://unix.stackexchange.com/questions/15826/io-redirection-and-the-head-command
head -n 91 plot_compare_reduction.py > tmp.py
mv tmp.py plot_compare_reduction.py
git add plot_compare_reduction.py
git commit -m 'removing part corresponding to Caching transformers within a Pipeline'

Did we do what we wanted? `git diff` is your ally because it will allow you to see what exactly changed relative to a past commit:

In [ ]:
%%bash
cd my_first_repo
git diff `git log --pretty=format:'%h' -n 1`~1

The `diff` output shows that we managed to do what we wanted.

Note that here we used a trick:
```
`git log --pretty=format:'%h' -n 1`~1
```
to retrieve the previous' commit sha without having to know the sha in advance. Indeed:

In [ ]:
%%bash
cd my_first_repo
git log --pretty=format:'%h' -n 1

Returns the short sha of last commit (the one we just did), and by adding the `~1` suffix, we get a reference to the one previous to the last. 

In fact, in this particular case we overcomplicated the syntax since we could have simply used `git diff HEAD~1` to get the information we wanted.

**Exercise 1.4**: You will now modify the `my_first_repo/plot_compare_reduction.py` file so that:
1. The values in the variable `C_OPTIONS` are `[1, 10, 100, 500]`
2. Instead of plotting a figure, it saves the chart in a PNG file (hint: [plt.savefig](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.savefig.html))

It's important that you stage and commit **each of changes separately**. Make sure that you accomplished what you wanted by:
1. Checking each diff relative to each previous commit.
2. Running the resulting code

## How to maintain different versions of a file without going crazy (branch, checkout)

As you know, the typical data science flow involves a lot more changes than the ones we performed in Exercise 1.4. In particular, the development flow is not linear: one wants to be able to go back and forth between different experiments, and try different variations of a given change before consolidating it.

The VCS-free approach to dealing with this variability is to append suffixes to our code files and artifacts (e.g. charts), which typically ends up with a pletora of files with cryptic names, each corresponding to a different variation:
* plot_compare_reduction_v2.py
* plot_compare_reduction_v2_include_LR.py
* plot_compare_reduction_iris_data_v2.py...

You get the point. The situation becomes even more daunting when there are several people working on the same project.

Luckily, Git gives a number of tools to support this type of work in cleaner and more powerful way, leveraging the power of `branches`. As we saw during the lecture, one can think of a branch as a diversion from the master branch in such a way that we can always go back to any other branch (including the master one) with just a few keystrokes.

Let's get to it. Suppose that we want to experiment with some serious changes to the `my_first_repo/plot_compare_reduction.py` script, changes that we are not sure will remain permanent. The first step is to create a new branch where we will start commiting these changes into, so that the stuff we have in our master branch remains unnaffected:

In [ ]:
%%bash
cd my_first_repo/
# I usually use the convention of giving my branches names following the convention {user}/{short description}
git branch -c $USER/experiment

I now have two branches in my repository, as `git branch` cordially explains:

In [ ]:
%%bash
cd my_first_repo/
git branch

The * indicates to me that I'm currently working on my `master` branch. Since I want to perform a few naughty changes, I need first to switch to the newly created branch, and for that I will use `git checkout`:

In [ ]:
%%bash
cd my_first_repo/
git checkout $USER/experiment

**Exercise 1.5**: Can you visualize the differences between those two branches? (the `$USER/experiment` branch and the `master` branch) (hint: you can use git diff)

To make things more interesting, we will create yet another branch to work on a couple changes that we would like to peform in order to show confidence intervals in the bar chart of our comparison plot:

In [ ]:
%%bash
cd my_first_repo/
# `git checkout -b` performs a `branch -c` + `checkout`
git checkout -b $USER/add_cis_to_plot master

**Exercise 1.6**: Let's practice moving in and out of branches... it's a bit of work but we'll use what you do here later to work on `merge` and `rebases`. Please do the following:
1. Make a few changes in the experiment code (for example: try another classifier or dimensionality reduction method) while working on the `$USER/experiment` branch. 
2. Introduce confidence intervals to the bar charts while working on the `$USER/add_cis_to_plot` branch

**Important**: make sure you develop each change in the right branch, and that you stage and commit your work incrementally. Incremental commits are in general preferable over big long ones.

*Questions*:
1. After you've made the changes, what are the differences between the `$USER/experiment` and the `$USER/add_cis_to_plot` branches? 
2. Are there any conflicting changes?

## Time-travel with git (checkout, revert, reset)

<img src="time_travel.jpg" height="240" align="center"/>

During the course of the past exercises you have probably encountered a few situations where you made a mistake:
* Perhaps you mistakenly staged files that you don't really want to track
* Or you commited a change that you now regret
* Or simply you want to go back to a state where you know things were stable...

Don't you worry! The ability to go back and forth in your development timeline is one of the most powerful features git has to offer. 

### Unstaging changes
One of the most common situations is when you accidentally stage changes on a tracked file, or start tracking a file you didn't want to track. Let's simulate this situation in our playground repo:

In [ ]:
%%bash
cd my_first_repo/
# we will switch to a new branch for convenience
git checkout -b $USER/time_travel master
# accidental removal of "a few" lines in `plot_compare_reduction.py`
head -n 50 plot_compare_reduction.py > plot_compare_reduction.py.tmp
mv plot_compare_reduction.py.tmp plot_compare_reduction.py
# change we actually want to make:
echo "import sys\npython_major_version=sys.version_info[0]\nassert python_major_version==3" >> test.py
# we carelessly add the accidental changes and the one we want to make
git add --all
git status

The first problem is that we have added a `.png` file which is the artifact of our computation in `plot_compare_reduction.py`, and as we discussed earlier, we don't generally want to check artifacts (binaries) into git. So we will follow `git`'s suggestion and `unstage` it, by applying:

In [ ]:
%%bash
cd my_first_repo/
git reset HEAD plot_compare_reduction.png
git status

We have successfully solved the first problem: the addition of an unwanted new file. But how did this happened? 
    
A proper response to this answer would require a level of understanding of the git internals and commands that is a bit beyond the scope of this course, but I will attempt to simplify the explanation here. When we user `git reset` on a given path (file), what we told git is to bring the state of that file **in the staging area** (the index) to the commit `HEAD` is pointing too (the last commit of the branch). Since that commit points to a snapshot of the repository where the file `plot_compare_reduction.png` doesn't exist, the result is that the `add` operation we just performed on it is undone, so the file goes back to the pile of `untracked files`, as the `git status` command states.

More information on `git reset`:

* https://stackoverflow.com/questions/3639342/whats-the-difference-between-git-reset-and-git-checkout

* https://git-scm.com/book/en/v2/Git-Tools-Reset-Demystified

We still have one more problem to solve, which is the accidental deletion of content from `plot_compare_reduction.py`. Since we know that the state of this path in HEAD is the correct one, we can apply the same operation:

In [ ]:
%%bash
cd my_first_repo/
git reset HEAD plot_compare_reduction.py
git status

The accidental change is no longer staged for commit (it no longer exists in the index/staging area), but if we look at the file, it still has the changes we performed in the working tree (our filesystem). In order to undo the changes we did in the file in our working tree, we need to perform an additional operation of restoring the state of the file to the one we had stored in HEAD:

In [ ]:
%%bash
cd my_first_repo/
git checkout HEAD plot_compare_reduction.py
git status

Note that we could also have used the command `git checkout -- plot_compare_reduction.py`, where `--` in this case indicates `checkout` to bring the path to the state in HEAD.

So we have managed to succesfully unstage the accidental changes, so now we can merrily commit our desired changes and move on:

In [ ]:
%%bash
cd my_first_repo/
git commit -m 'adding python version check'
git status

### Reverting previous work

Another common situation occurs when we introduce a change in our repository that we later regret. Contrary to the previous situation, in this case we have already committed the change.

If the unwanted change is the last one we committed, we could use `git reset` to start working from the last stable state. Unfortunately, this would mean that we're "re-writing history" which can cause problems if the changes have already been shared with other collaborators, because each one of us will have a different version of history.

So a safer way is to use `git revert`, which allows us to undo the changes in a specific commit by adding an additional commit that does just as much:

In [ ]:
%%bash
cd my_first_repo/
git log -n 5

Suppose we want to undo the changes we performed earlier in commit "Updating C_OPTIONS to desired values"

In [ ]:
%%bash
cd my_first_repo/
# update this commit SHA to the one you see in your commit log
git show XXXX # <---

Reverting to the state previous to these changes is a piece of cake:

In [ ]:
%%bash
cd my_first_repo/
git revert XXXX # update this commit SHA to the one you see in your commit log

And you're done: you can verify by yourself that the state of `plot_compare_reduction.py` is the one previous to the changes we performed.

**Warning**: Note that revert gives you a quick way of undoing any type of changes, not just simple changes affecting a single file. This doesn't mean reverting is without risks. Consider for example the setup of the following exercise.

**Exercise 1.7**: We will now create a situation where reverting can create a mess. Please follow the next steps of havoc-making:
0. Create and checkout new branch off master called `revert_havoc`
1. Create a new python module `dependency.py` with a function named `my_dependency()`. Stage and commit the changes.
2. Now make a call to that function somewhere in `plot_compare_reduction.py` (Don't forget to import the module first). Stage and commit those changes.
3. Now revert the commit created in Step 1. What changes does it induce? Will your code work after the revert? 

### Summary of time-travel operations

In this last section we have learned how to time-travel: go back in time to a state where we are more comfy. As you probably suspected, time-travel is not without risk so it has to be used carefully and judiciously. 

A nice and concise summary of the time-travel operations is the following:

| Command |	Scope |	Common use cases|
|---|---|---|
|git reset|Commit-level|Discard commits in a private branch or throw away uncommited changes|
|git reset|File-level|Unstage a file|
|git checkout|Commit-level|Switch between branches or inspect old snapshots|
|git checkout|File-level|Discard changes in the working directory|
|git revert|Commit-level|Undo commits in a public branch|
|git revert|File-level|(N/A)|

(Adapted from https://www.atlassian.com/git/tutorials/resetting-checking-out-and-reverting)